In [ ]:
#classifying news using bidirectional LSTM

In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
#importing the dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/news/train 2.csv')

In [ ]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(18285, 5)

In [ ]:
#creating x and y
x=df.drop('label',axis=1)
y=df.iloc[:,-1]

In [ ]:
print(x.shape)

(18285, 4)


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
#defining vocab size 
#10000
voc_size=10000

In [ ]:
#one hot rep
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages=x.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
#data preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
messages.shape

(18285, 5)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
#one hot rep
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[6714, 3656, 9329, 7121, 7201, 8974, 9467, 1445, 1038, 3768],
 [4462, 8788, 2289, 3044, 949, 2977, 2690],
 [9231, 8315, 9137, 5659],
 [5093, 264, 1990, 1070, 1624, 3884],
 [9776, 949, 2849, 5065, 7080, 6528, 949, 8330, 4811, 545],
 [8593,
  6586,
  7927,
  9307,
  7584,
  2405,
  6672,
  3821,
  6941,
  336,
  6227,
  3784,
  1853,
  6538,
  2690],
 [6983, 5845, 8016, 1110, 1602, 6161, 311, 4794, 9954, 6893, 2066],
 [5921, 5717, 1860, 203, 7279, 3706, 2405, 9253, 9954, 6893, 2066],
 [7633, 7873, 9900, 8748, 8786, 7370, 3661, 9697, 2405, 4790],
 [6197, 5017, 3759, 2782, 1421, 6574, 7809, 6435],
 [7835, 7030, 9430, 5325, 5200, 4473, 8737, 3004, 4508, 1413, 4171],
 [1070, 702, 7201, 7370, 2405, 7279],
 [1827, 4246, 9256, 6205, 1028, 3135, 3995, 3174, 723],
 [5569, 9255, 4926, 4762, 9905, 173, 7499, 9954, 6893, 2066],
 [5760, 5673, 8502, 318, 1412, 9954, 6893, 2066],
 [2065, 4612, 453, 9462, 108, 2595, 1929, 914, 9295, 7501],
 [5715, 3838, 8788],
 [4467, 2618, 664, 8823, 2405, 6271, 5935,

In [ ]:
#embedding
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1445 1038 3768]
 [   0    0    0 ...  949 2977 2690]
 [   0    0    0 ... 8315 9137 5659]
 ...
 [   0    0    0 ... 9954 6893 2066]
 [   0    0    0 ... 6503 8445 5277]
 [   0    0    0 ... 3497 1280 1253]]


In [ ]:
embedded_docs.shape

(18285, 20)

In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
#splitting
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 21s 77ms/step - loss: 0.3060 - accuracy: 0.8588 - val_loss: 0.1989 - val_accuracy: 0.9150
Epoch 2/10
192/192 [==============================] - 13s 69ms/step - loss: 0.1213 - accuracy: 0.9566 - val_loss: 0.2066 - val_accuracy: 0.9148
Epoch 3/10
192/192 [==============================] - 15s 80ms/step - loss: 0.0744 - accuracy: 0.9741 - val_loss: 0.2265 - val_accuracy: 0.9155
Epoch 4/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0488 - accuracy: 0.9837 - val_loss: 0.3121 - val_accuracy: 0.9105
Epoch 5/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0330 - accuracy: 0.9905 - val_loss: 0.3611 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 15s 78ms/step - loss: 0.0241 - accuracy: 0.9926 - val_loss: 0.4473 - val_accuracy: 0.9051
Epoch 7/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0181 - accuracy: 0.9950 - val_loss: 0.4796 - val_accuracy:

In [ ]:
y_pred1=model1.predict(X_test)

189/189 [==============================] - 6s 15ms/step


In [ ]:
#defining classifier function
def classifier(y_pred):
  for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
      y_pred[i]=1
    else:
      y_pred[i]=0
  return y_pred 

In [ ]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

In [ ]:
yp=classifier(y_pred1)

In [ ]:
yp

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)